# Options IV Calculation Test

This notebook tests the options data fetching and implied volatility (IV) calculation pipeline.


In [9]:
from adapters.options_adapter import OptionsAdapter
from adapters.rates_adapter import RatesAdapter
from adapters.ticker_adapter import TickerAdapter
from models.options_data import OptionsRequest, OptionType
from datetime import date
from dateutil.relativedelta import relativedelta
from engines.IV_smile import IVEngine


## Setup

Initialize the adapter and create the options request.


In [10]:
adapter = OptionsAdapter()

# Simulate duration selection (e.g. '1y' from frontend)
today = date.today()
expiry_end = today + relativedelta(years=1)

req = OptionsRequest(
    ticker="AAPL",
    optionType=OptionType.CALL,
    expiryEnd=expiry_end,
    strikeMin=200,
    strikeMax=300,
    limit=500,
)


## Fetch Options Data


In [12]:
print(f"Requesting options for {req.ticker} until {expiry_end}...")
df = adapter.fetch_option_chain(req)
print(f"Fetched {len(df)} contracts.")



Requesting options for AAPL until 2026-12-04...
Fetched 329 contracts.


## Display Options Data


In [16]:
df[["optionType", "strike", "timeToExpiry", "midPrice"]]

,optionType,strike,timeToExpiry,midPrice
0,call,200.0,0.004196,85.600
1,call,205.0,0.004196,78.540
2,call,210.0,0.004196,73.720
3,call,215.0,0.004196,70.825
4,call,220.0,0.004196,64.665
...,...,...,...,...
324,call,280.0,0.790384,33.820
325,call,285.0,0.790384,30.700
326,call,290.0,0.790384,27.565
327,call,295.0,0.790384,25.775


## Calculate Implied Volatility


In [21]:
if not df.empty:
    rate = RatesAdapter.getTodayRate() or 0.045
    base_info = TickerAdapter.fetchBasic(req.ticker)
    div = base_info.dividendYield
    spot = base_info.spot
    
    print(f"Spot price: ${spot:.2f}")
    print(f"Risk-free rate: {rate:.4f}")
    print(f"Dividend yield: {div:.4f}")
    print("\nCalculating IVs...")
    
    surface_data = IVEngine.generateIVSmile(df, rate, div, spot, OptionType.CALL)
    
    print("\nIV Calculation Results:")
    surface_data


Spot price: $286.11
Risk-free rate: 0.0395
Dividend yield: 0.3700

Calculating IVs...

IV Calculation Results:


## Optional: Generate Surface Mesh

Uncomment the code below to generate a surface mesh (may be verbose).


In [15]:
# surface_data = IVEngine.generate_surface_mesh(df, grid_size=30)
# print(surface_data) # Too verbose
